$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

$\textbf{Preliminaries}$ 

The libraries used here are:

In [1]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

In [2]:
from Uniandes_Framework.delphes_reader import DelphesLoader #The class that allows us to access all paths of delphes files inside the server.
from Uniandes_Framework.delphes_reader import classifier #It contains functions to classify particles.
from Uniandes_Framework.delphes_reader import root_analysis #It contains some useful functions like make_histograms or get_kinematics_row.
from Uniandes_Framework.delphes_reader import Quiet #Context manager for silencing certain ROOT operations.

from ROOT import TChain #It is necessary to read .root files.

import pandas as pd #Python library is useful for data science.

Welcome to JupyROOT 6.22/06


With the objective of learning how to use Uniandes_Framework to read delphes files, we will reconstruct Z Boson Mass for some events:

**1. Get delphes files (.root) paths.**

First, we must get delphes file paths of the signal. In this case, the signal is Z Boson. To do this, we can use DelphesLoader.

### It is necessary to move Z_Tutorial folder to MadGraph_Simulations

In [3]:
# DelphesLoader('Z_Tutorial', os.path.join(os.path.dirname(os.getcwd()), 'SimulationsPaths.csv')).Forest

These are all the root files of Z on the hep-server2. Now, we can save this list in a variable named Path_Signal:

In [4]:
Paths_Signal = DelphesLoader('zz', os.path.join(os.path.dirname(os.getcwd()), 'SimulationsPaths.csv')).Forest[:6] 

zz imported with 200 trees!
/Madgraph_Simulations/SM_Backgrounds/zz/


Now, we must get delphes file paths of the background. For simplicity we will consider just 'ww', 'ttbar', 'stop' as background:

In [5]:
Paths_WW = DelphesLoader('ww').Forest[:6] 
Paths_ttbar = DelphesLoader('ttbar').Forest[:6]
Paths_stop = DelphesLoader('stop').Forest[:6]

ww imported with 250 trees!
/Madgraph_Simulations/SM_Backgrounds/ww/
ttbar imported with 500 trees!
/Madgraph_Simulations/SM_Backgrounds/ttbar/
stop imported with 232 trees!
/Madgraph_Simulations/SM_Backgrounds/stop


We can save all of those paths (signal and background) in one dictionary:

In [6]:
Paths = {'z':Paths_Signal, 'ww': Paths_WW, 'ttbar': Paths_ttbar, 'stop': Paths_stop}

In [7]:
Paths

{'z': ['/Madgraph_Simulations/SM_Backgrounds/zz/zz_1/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/zz/zz_2/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/zz/zz_3/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/zz/zz_4/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/zz/zz_5/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/zz/zz_6/Events/run_01/m_delphes_events.root'],
 'ww': ['/Madgraph_Simulations/SM_Backgrounds/ww/ww_1/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/ww/ww_2/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/ww/ww_3/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/ww/ww_4/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/ww/ww_5/Events/run_01/m_delphes_events.root',
  '/Madgraph_Simulations/SM_Backgrounds/ww

At this point, we can access all delphes file paths.

**2. Extract information from root files to create .CSV files:**

To do this, we have to read those .root files. 

If we want to read .root files of signal we have to run the following code:

In [8]:
with Quiet(): #Context manager for silencing certain ROOT operations.
    
    for Path in Paths['z']: #Paths[z] is the list with the paths of signal z.

        #To read a .root file we have to create a Tree in ROOT and associate it the path.
        tree = TChain("Delphes;1") #Empty TChain of ROOT.
        tree.Add(Path) #Now we associate tree with the path of a .root file.

        #At this point we must go over all events in the tree, so:
        for event in tree:
            #Here is where we will use Uniandes_Framework to extract information
            break
        break

This is a very simple code. Before reconstruy Z Boson, we can use Uniandes_Framework to extract muons and electron particles and test some framwork's functions:

In [9]:
#We have to create a dictionary to save the muons and the electrons.
Dictionary = {}

with Quiet(): #Context manager for silencing certain ROOT operations.
    
    for Path in Paths['z']: #Paths[z] is the list with the paths of signal z.
        
        Dictionary['muon'] = [] #We have to create a list for muons.
        Dictionary['electron'] = [] #We have to create a list for muons.
            
        #To read a .root file we have to create a Tree in ROOT and associate it the path.
        tree = TChain("Delphes;1") #Empty TChain of ROOT.
        tree.Add(Path) #Now we associate tree with the path of a .root file.

        #At this point we must go over all events in the tree, so:
        for event in tree:
            #Now we can use Uniandes_Framework to extract information:

            for particle in classifier.get_muons(event): #clasificator.get_muons(event) is a list with muons in the event
                Dictionary['muon'].append(particle)
                
            for particle in classifier.get_electrons(event): #clasificator.get_electrons(event) is a list with electrons in the event
                Dictionary['electron'].append(particle)                
        break

In [10]:
Dictionary.keys()

dict_keys(['muon', 'electron'])

**What contain Dictionary['muons'] and Dictionary['electrons']?**

In [11]:
len(Dictionary['muon']) #It is a list with 83754 elements that are objects of particle class.

4806

In [12]:
len(Dictionary['electron']) #It is a list with 15 elements that are objects of particle class.

3618

We can access all kinematic information of any particle. 

**For example:**

In [13]:
Dictionary['muon'][0].Charge #Charge of muon 0.

1.0

In [14]:
Dictionary['electron'][1].pt #Transversal momentum of electron 1.

37.16234970092773

In [15]:
Dictionary['muon'][2].GetTLV() #TLV of muon 2.

We can also extract main kinematic variables using the function **root_analysis.get_kinematics_row**:

In [16]:
root_analysis.get_kinematics_row([Dictionary['muon'][0]])

{'pT_{#mu_{1}}(GeV)': 36.52714538574219,
 '#eta_{#mu_{1}}': 0.6355732679367065,
 '#phi_{#mu_{1}}': 0.4256139993667603,
 'Energy_{#mu_{1}}(GeV)': 44.156623621061925,
 'Mass_{#mu_{1}}(GeV))': 0.10565837549941551}

We can use this function to create a .csv with **root_analysis.generate_csv:**

In [17]:
muon_list = [root_analysis.get_kinematics_row([Dictionary['muon'][0]])]

root_analysis.generate_csv(muon_list, 'Data_muon.csv')

In [18]:
pd.read_csv('Data_muon.csv')

,pT_{#mu_{1}}(GeV),#eta_{#mu_{1}},#phi_{#mu_{1}},Energy_{#mu_{1}}(GeV),Mass_{#mu_{1}}(GeV))
0,36.527145,0.635573,0.425614,44.156624,0.105658


In addition to this, if we want to get all the particles without a label it is enough to use the following function:

In [19]:
Unified = classifier.get_unified(Dictionary)

In [20]:
Unified.keys()

dict_keys(['all'])

In [21]:
len(Unified['all']) #it is a list with 83769 elements that are objects of particle class (83754 muons + 15 electrons).

8424

Furthermore, if we want to extract the particles that are within the range of kinematic cuts, we can use clasificator.get_good_particles or clasificator.get_good_leptons.

### It is necessary to add an example of get_good methods.

With this in mind, now we can use Uniandes_Framework to reconstruct Z Boson:

In [22]:
#At first, we have to create a dictionary to save the z reconstructed kinematic information of 'z', 'ww', 'ttbar', 'stop' signals:
Z_reconstructed_particles = {}

with Quiet(): #Context manager for silencing certain ROOT operations.
    
    for key in Paths.keys(): #Paths.keys() is ['z', 'ww', 'ttbar', 'stop']
        
        Z_reconstructed_particles[key] = [] #We have to create a list to save the kinematic information of z reconstructed for each key.
        
        for Path in Paths[key]: #Paths[key] is the list with the delphes file paths.
            
            #To read a .root file we have to create a Tree in ROOT and associate it the path.
            tree = TChain("Delphes;1") #Empty TChain of ROOT.
            tree.Add(Path) #Now we associate tree with the path of a .root file.

            #At this point we must go over all events in the tree, so:
            for event in tree:
                #Now we can use Uniandes_Framework to extract information:
                
                muons = classifier.get_muons(event) #clasificator.get_muons(event) is a list that contains the muons presents in the event.
                
                #We need at least 2 muons in this event to reconstruct Z Boson, so:
                if (len(muons) < 2): continue 
                
                #How we can reconstruct Z Boson? We have to sum the TLorentz vector of the muon pair: 
                Z = muons[0].GetTLV() + muons[1].GetTLV()
                
                #However, Z is not an object of particle class, it is a TLV. So, we can not use root_analysis.get_kinematics_row().  
                #If we want his kinematic information we have to create the dictionary by ourselves:
                kinematic_information =   {"pT_{Z}(GeV)": Z.Pt(), 
                                           "#eta_{Z}": Z.Eta(), 
                                           "#phi_{Z}": Z.Phi(), 
                                           "Energy_{Z}(GeV)": Z.Energy(), 
                                           "Mass_{Z}(GeV)" : Z.M()} 
                #This dictionary has the same structure of an get_kinematics_row() output. 

                #In addition to this, we can add to this dictionary the kinematic information of the two muons using get_kinematics_row().
                kinematic_information |= root_analysis.get_kinematics_row(muons[:2])
                
                #Finally, we can save Z_kinematic_information for each event in a list:
                Z_reconstructed_particles[key].append(kinematic_information)  
            break
        
        #At this point we have Z_reconstructed_particles[key] (a list) filled with kinematic information, so we must save it:
        root_analysis.generate_csv(Z_reconstructed_particles[key], f'Data_{key}.csv')

So,

In [23]:
pd.read_csv('Data_z.csv')

,pT_{Z}(GeV),#eta_{Z},#phi_{Z},Energy_{Z}(GeV),Mass_{Z}(GeV),pT_{#mu_{1}}(GeV),#eta_{#mu_{1}},#phi_{#mu_{1}},Energy_{#mu_{1}}(GeV),Mass_{#mu_{1}}(GeV)),...,#Delta{#eta}_{#mu_{1}#mu_{2}},#Delta{#phi}_{#mu_{1}#mu_{2}},#Delta{pT}_{#mu_{1}#mu_{2}}(GeV),#Delta{#vec{pT}}_{#mu_{1}#mu_{2}}(GeV),#Delta{#vec{p}}_{#mu_{1}#mu_{2}}(GeV),pT_{#mu_{2}}(GeV),#eta_{#mu_{2}},#phi_{#mu_{2}},Energy_{#mu_{2}}(GeV),Mass_{#mu_{2}}(GeV))
0,6.670549,-1.329340,-0.236722,92.415652,91.426488,36.527145,0.635573,0.425614,44.156624,0.105658,...,1.624645,3.011141,4.992168,67.918171,91.518237,31.534977,-0.989072,-2.585527,48.259029,0.105658
1,74.410640,1.126648,0.171202,146.960482,74.208303,71.201958,1.091211,-0.208553,117.970546,0.105658,...,0.782697,-1.646518,43.539167,78.313180,115.863612,27.662790,0.308513,1.437965,28.989936,0.105658
2,48.951196,-1.636438,-2.515581,157.805546,88.738027,65.459869,-0.698063,-2.850710,82.067234,0.105658,...,1.071634,-2.444619,40.378736,86.208444,88.963183,25.081133,-1.769697,-0.406091,75.738312,0.105658
3,27.238409,-2.362710,-0.876849,171.561237,90.234615,33.655849,-2.189325,-1.467443,152.144212,0.105658,...,-2.454177,-2.199601,14.900772,47.191178,160.495319,18.755077,0.264853,0.732159,19.417025,0.105658
4,40.073853,0.515274,-1.322458,101.510940,90.736308,39.661335,1.098369,-0.903138,66.089465,0.105658,...,2.489354,1.756148,23.061361,45.730161,95.778695,16.599974,-1.390985,-2.659286,35.421475,0.105658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,70.262849,0.158368,-3.070388,115.979402,91.594191,45.507664,-0.777255,-3.036240,59.960054,0.105658,...,-2.230840,0.096763,20.677288,20.931366,91.678679,24.830376,1.453585,-3.133003,56.019348,0.105658
1551,29.194155,-0.484478,-1.200406,92.490289,86.521406,55.525505,-0.211766,-0.596170,56.775277,0.105658,...,-0.131592,2.656943,19.925129,88.592541,89.047425,35.600376,-0.080174,3.030072,35.715012,0.105658
1552,70.447016,-1.162722,0.773404,152.393062,89.034649,71.944191,-0.390165,0.530081,77.490061,0.105658,...,1.752208,-1.778244,54.599535,77.400423,89.071976,17.344656,-2.142373,2.308325,74.903001,0.105658
1553,20.864963,-1.497754,2.237081,102.261882,89.767104,32.237865,0.565116,2.037325,37.524160,0.105658,...,2.893933,-2.803822,19.744114,44.219916,93.801206,12.493752,-2.328817,-1.442038,64.737723,0.105658


In [24]:
pd.read_csv('Data_ww.csv')

,pT_{Z}(GeV),#eta_{Z},#phi_{Z},Energy_{Z}(GeV),Mass_{Z}(GeV),pT_{#mu_{1}}(GeV),#eta_{#mu_{1}},#phi_{#mu_{1}},Energy_{#mu_{1}}(GeV),Mass_{#mu_{1}}(GeV)),...,#Delta{#eta}_{#mu_{1}#mu_{2}},#Delta{#phi}_{#mu_{1}#mu_{2}},#Delta{pT}_{#mu_{1}#mu_{2}}(GeV),#Delta{#vec{pT}}_{#mu_{1}#mu_{2}}(GeV),#Delta{#vec{p}}_{#mu_{1}#mu_{2}}(GeV),pT_{#mu_{2}}(GeV),#eta_{#mu_{2}},#phi_{#mu_{2}},Energy_{#mu_{2}}(GeV),Mass_{#mu_{2}}(GeV))
0,41.450595,2.246616,0.309652,223.867783,104.153360,60.238182,1.383534,-0.692575,127.694849,0.105658,...,0.148018,-2.397226,8.682526,104.187796,108.818697,51.555656,1.235516,1.704651,96.172934,0.105658
1,59.747003,2.090162,0.971970,295.158432,164.215993,106.887489,1.328515,0.477510,215.929003,0.105658,...,0.580007,-2.660349,45.633564,163.658954,213.667080,61.253925,0.748508,3.137858,79.229430,0.105658
2,22.288453,2.348517,-0.077184,179.500828,135.486983,47.394806,1.785429,-0.337599,145.261789,0.105658,...,2.532970,-2.923193,20.907772,73.477112,175.164862,26.487034,-0.747542,2.585595,34.239040,0.105658
3,39.584572,-1.851892,-0.810271,139.905349,53.625679,37.521748,-1.066819,-1.462149,60.977107,0.105658,...,0.759795,-1.817748,12.755196,49.755291,56.550085,24.766552,-1.826614,0.355599,78.928242,0.105658
4,20.076604,-1.986348,2.568742,149.230630,129.278067,49.605492,-1.431904,2.623706,109.765002,0.105658,...,-2.228257,3.104298,20.025717,79.172380,147.155618,29.579775,0.796353,-0.480592,39.465628,0.105658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,46.937619,-1.306490,1.418606,126.232366,85.323836,37.399624,-1.511820,0.528789,88.926729,0.105658,...,-1.521461,-1.782811,0.095871,58.115950,99.723867,37.303753,0.009642,2.311600,37.305636,0.105658
287,76.068378,2.201324,-0.063515,348.104478,11.246892,60.394993,2.146950,-0.125541,261.980047,0.105658,...,-0.206300,-0.294828,44.167717,45.114977,176.214772,16.227276,2.353250,0.169287,86.124431,0.105658
288,76.795021,1.268362,0.899658,152.407104,39.104396,47.219013,1.311121,0.502784,93.962297,0.105658,...,0.312903,-0.898944,9.292236,37.929882,52.826140,37.926777,0.998217,1.401728,58.444808,0.105658
289,71.144513,-1.671477,0.118981,236.686245,132.782604,50.245888,-1.997228,-0.623441,188.530468,0.105658,...,-1.979637,-1.525593,2.097675,68.001893,193.225849,48.148212,-0.017591,0.902152,48.155778,0.105658


Until now we are just considering the first root file of each signal (we had a break inside the second for) to guarantee that the tutorial does not take much time of running. However, If you want to consider all the root files of each signal. It is necessary to delete the "break" inside the second for. This will be done in **4_Final_Project.ipynb**.

**That would take a long time, of course.**